# CHUNKING

In [1]:
import os
import json
import re
import tiktoken
from docx import Document
from PyPDF2 import PdfReader
import nltk
from nltk.tokenize import word_tokenize
import pdfplumber

In [2]:
def remove_law_header(text):
    """
    Tìm "Chương I" và lấy từ đó trở đi (Điều 1 nằm TRONG Chương I).
    """
    pattern = r"(Chương\s+[I1])"
    match = re.search(pattern, text, flags=re.IGNORECASE)

    if match:
        # Lấy từ "Chương I" trở đi (Điều 1 nằm sau đó)
        return text[match.start():]
    else:
        # Không tìm thấy Chương I => giữ nguyên
        return text

In [3]:
def extract_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def clean_text(t):
    """Giữ lại newline để regex tách điều"""
    t = t.replace("\xa0", " ")
    t = re.sub(r"[ \t]+", " ", t)  # ✅ Chỉ compress space/tab, giữ \n
    return t.strip()


In [4]:
#Tách điều-khoản

regex_dieu = re.compile(r"(?m)^(Điều\s+\d+)\.")   # dạng: "Điều 1."
regex_khoan = re.compile(r"(?<=\n|\s)(\d{1,2})\.(?=\s)")   # dạng: "1. "
regex_diem = re.compile(r"^[a-z]\)$")  

def split_by_dieu(text):
    parts = regex_dieu.split(text)
    results = []
    
    for i in range(1,len(parts), 2):
        if i + 1 < len(parts):
            dieu_title = parts[i].strip()
            dieu_content = parts[i+1].strip()
            results.append((dieu_title, dieu_content))

    return results

def split_by_khoan(dieu_content):
    parts = regex_khoan.split(dieu_content)
    results = []

    for i in range(1, len(parts), 2):
        khoan_num = parts[i]
        khoan_content = parts[i+1].strip()
        results.append((khoan_num, khoan_content))

    return results


def split_by_diem(khoan_content):
    parts = regex_diem.split(khoan_content)
    results = []

    for i in range(1, len(parts), 2):
        diem_letter = parts[i]
        diem_content = parts[i+1].strip()
        results.append((diem_letter, diem_content))

    return results

In [5]:
#Đo token

def count_tokens(text):
    return len(text.split())

In [6]:
def create_chunks(law_name, text):
    chunks = []
    ds_dieu = split_by_dieu(text)

    for dieu_title, dieu_body in ds_dieu:
        # ✅ FIX: Extract số điều từ "Điều 2" chứ không phải từ toàn dieu_title
        nums = re.findall(r"\d+", dieu_title)
        if not nums:
            continue
        dieu_num = int(nums[0])  # ← Đây phải là 1, 2, 3, 4...

        ds_khoan = split_by_khoan(dieu_body)

        if not ds_khoan or len(ds_khoan) == 0:
            chunk_id = f"{law_name}_d{dieu_num}"
            chunks.append({
                "id": chunk_id,
                "law": law_name,
                "dieu": dieu_num,  # ← Sẽ là 2, 3, 4... chứ không phải luôn 1
                "khoan": None,
                "diem": None,
                "text": f"[Điều {dieu_num}]: {dieu_body.strip()}"
            })
            continue

        for khoan_num, khoan_body in ds_khoan:
            chunk_id = f"{law_name}_d{dieu_num}_k{khoan_num}"
            
            chunks.append({
                "id": chunk_id,
                "law": law_name,
                "dieu": dieu_num,  # ← Sẽ là 2, 3, 4... chứ không phải luôn 1
                "khoan": int(khoan_num),
                "diem": None,
                "text": f"[Điều {dieu_num}] [Khoản {khoan_num}]: {khoan_body.strip()}"
            })

    return chunks

In [7]:
if __name__ == "__main__":
    PDF_PATH = "luatgtdb.pdf"        
    LAW_NAME = "35/2024/QH15"

    raw_text = extract_text(PDF_PATH)
    clean = clean_text(raw_text)

    chunks = create_chunks(LAW_NAME, clean)

    print("Tổng chunk tạo được:", len(chunks))

    with open("luatgtdb_chunks.json", "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    print("Đã lưu file luatgtdb_chunks.json ✔")

Tổng chunk tạo được: 338
Đã lưu file luatgtdb_chunks.json ✔


In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch


In [27]:
import gradio as gr

In [ ]:
import os
from operator import itemgetter
import gradio as gr
import json

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings  
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
# CẤU HÌNH API KEY GEMINI
os.environ["GOOGLE_API_KEY"] = "YOUR_API_HERE"  

In [ ]:
# KHỞI TẠO LLM & EMBEDDING
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
)

embeddings = GoogleGenerativeAIEmbeddings(
     model="models/text-embedding-004"
 )

In [ ]:
# LOAD JSON & TẠO DOCUMENTS
JSON_PATH = "luatgtdb_chunks.json"   

with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw_chunks = json.load(f)   # list[dict]

documents = [
    Document(
        page_content=item["text"],
        metadata={
            "id": item.get("id"),
            "law": item.get("law"),
            "dieu": item.get("dieu"),
            "khoan": item.get("khoan"),
            "diem": item.get("diem"),
        },
    )
    for item in raw_chunks
]

print(f"Đã load {len(documents)} chunks từ JSON.")

In [ ]:
# TẠO VECTORSTORE & RETRIEVER

vectorstore = DocArrayInMemorySearch.from_documents(
    documents=documents,
    embedding=embeddings,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 30})
print("Vectorstore đã khởi tạo xong.")


In [ ]:
# ĐỊNH NGHĨA HÀM FORMAT CONTEXT

def format_docs(docs):
    parts = []
    for d in docs:
        law = d.metadata.get("law")
        dieu = d.metadata.get("dieu")
        khoan = d.metadata.get("khoan")
        diem = d.metadata.get("diem")
        header = f"[Luật {law} - Điều {dieu}, Khoản {khoan}"
        if diem:
            header += f", Điểm {diem}]"
        else:
            header += "]"
        parts.append(f"{header}\n{d.page_content}")
    return "\n\n---\n\n".join(parts)

In [ ]:
# PROMPT CHO RAG

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Bạn là trợ lý pháp lý, trả lời dựa trên các điều khoản luật trong phần context. "
            "Nếu có thể, hãy nêu rõ Điều/Khoản/Điểm."
        ),
        MessagesPlaceholder("chat_history"),  # lịch sử hội thoại nhiều turn
        (
            "human",
            "Câu hỏi: {question}\n\n"
            "Văn bản luật liên quan:\n{context}"
        ),
    ]
)

In [ ]:
#XÂY DỰNG RAG CHAIN

# Chain: question -> retriever -> format_docs -> prompt -> llm

base_rag_chain = (
    {
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
        "context": itemgetter("question")
        | RunnableLambda(lambda q: retriever.invoke(q))
        | RunnableLambda(format_docs),
    }
    | prompt
    | llm
)

Đã load 338 chunks từ JSON.
Vectorstore đã khởi tạo xong.


In [ ]:
_store = {} 
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in _store:
        _store[session_id] = InMemoryChatMessageHistory()
    return _store[session_id]


rag_with_history = RunnableWithMessageHistory(
    base_rag_chain,
    get_session_history,
    input_messages_key="question",      
    history_messages_key="chat_history"
)

In [29]:
def chat(message, history):
    """
    message: câu mới user gửi
    history: lịch sử hội thoại của Gradio (không cần tự xử lý, vì ta dùng InMemoryChatMessageHistory riêng)
    """
    # Dùng 1 session_id cố định cho Gradio. Nếu muốn multi-user thì map theo user id/cookie.
    session_id = "gradio-session"

    result = rag_with_history.invoke(
        {"question": message},
        config={"configurable": {"session_id": session_id}},
    )

    # ChatGoogleGenerativeAI trả về AIMessage -> dùng .content
    return result.content

In [30]:
view = gr.ChatInterface(
    fn=chat,
    type="messages",
    title="Luật 35/2024/QH15 Assistant"
).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


c:\Users\ngona\anaconda3\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)
